In [97]:
import pandas as pd
import sqlite3
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from  sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
import pickle   
import datetime
from sklearn.externals import joblib
import xgboost as xgb
import warnings
from sklearn import svm
import numpy as np
import tensorflow as tf
import peyton

warnings.filterwarnings("ignore")

warnings.filterwarnings("ignore", category=DeprecationWarning)

sns.set_style('whitegrid')
%matplotlib inline

con = sqlite3.connect('/home/Project_DS/Database/Sports_Results.db')
df = pd.read_sql_query("select * from ThroneHistorical;", con)
to_pred = pd.read_sql_query("select * from ThronePrediction;", con)
id_pred = to_pred.id
date_pred = to_pred.date

# Select current season and previous season
# Computate the season
now = datetime.datetime.now()

if now.month < 8:
    season_current = now.year - 1
    season_previous = now.year
if now.month > 8:
    season_current = now.year

season_previous = now.year - 1

df_previous = df[df['year'].astype(int) < 2017]

df['year'] = df['year'].astype(int)
df = df[df['year'].astype(int) >= season_previous].reset_index(drop = True) 


def preprocess_date(data_input, data_to_pred_input):
    
    data_to_pred = data_to_pred_input.copy()
    # Change to datetime and extract the hour
    data_input['date'] = pd.to_datetime(data_input['date'])
    data_input['hour'] = data_input['date'].apply(lambda x: x.hour)
    data_to_pred['date'] = pd.to_datetime(data_to_pred['date'])
    data_to_pred['hour'] = data_to_pred['date'].apply(lambda x: x.hour)
    
    # Drop datetime and date
    data_input.drop(['id','date','competition'], axis = 1, inplace = True)
    data_to_pred.drop(['id','competition','team_1_prob','team_2_prob','team_tie_prob','date','confidence'], axis = 1, inplace = True)
    
    # Label encoding
    to_stack = df[['team_1_name', 'team_2_name']]
    to_stack = pd.DataFrame(to_stack.stack()).reset_index(drop = True)[0]

    le = preprocessing.LabelEncoder()
    le.fit(to_stack)
    data_input['team_1_name'] = le.transform(data_input['team_1_name']) 
    data_input['team_2_name'] = le.transform(data_input['team_2_name'])
    data_input['day'] = data_input['day'].astype(int)
    data_input['month'] = data_input['month'].astype(int)

    data_to_pred['team_1_name'] = le.transform(data_to_pred['team_1_name']) 
    data_to_pred['team_2_name'] = le.transform(data_to_pred['team_2_name'])
    data_to_pred['day'] = data_to_pred['day'].astype(int)
    data_to_pred['month'] = data_to_pred['month'].astype(int)
    
    return data_input, data_to_pred

def fezture_to_predict(row):
    if row['team_1_score'] > row['team_2_score']:
        return 1
    if row['team_2_score'] > row['team_1_score']:
        return 2
    if row['team_1_score'] == row['team_2_score']:
        return 0

def aggreg_previous(data_input):
    data_input = data_input[data_input['year'] == 2016]
    
    return data_input

df, to_pred_ = preprocess_date(df, to_pred)
df['To_Predict'] = df.apply (lambda row: fezture_to_predict(row), axis=1)
df.drop(['team_1_score','team_2_score'], inplace = True, axis =1 )
df_previous['Result'] = df_previous.apply (lambda row: fezture_to_predict(row), axis=1)

X =  df.drop('To_Predict', axis = 1)
y =  df['To_Predict']

In [98]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

In [99]:
num_labels = 3

data = X.astype(np.float32)
labels = np.array(pd.get_dummies(y).astype(np.float32))
to_pred_ = to_pred_.astype(np.float32)

In [100]:
feature_size = data.shape[1]

graph = tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.constant(data)
    tf_train_labels = tf.constant(labels)

    weights = tf.Variable(tf.truncated_normal([feature_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels))

    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
    train_prediction = tf.nn.softmax(logits)



In [101]:
with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    for step in range(20001):
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if step % 500 == 0:
            print('step:{} loss:{:.6f} accuracy: {:.2f}'.format(step, l, accuracy(predictions, labels)))
    predict = tf.matmul(to_pred_, weights) + biases
    predictions =  tf.nn.softmax(predict)
    pred_df = pd.DataFrame(predictions.eval())

step:0 loss:2058.479980 accuracy: 22.82
step:500 loss:1027.226807 accuracy: 25.07
step:1000 loss:193.382278 accuracy: 27.18
step:1500 loss:7.382708 accuracy: 43.80
step:2000 loss:5.598506 accuracy: 46.97
step:2500 loss:4.279819 accuracy: 50.00
step:3000 loss:3.236464 accuracy: 50.66
step:3500 loss:2.464041 accuracy: 52.51
step:4000 loss:1.976426 accuracy: 52.51
step:4500 loss:1.591158 accuracy: 51.58
step:5000 loss:1.229086 accuracy: 51.58
step:5500 loss:0.984957 accuracy: 54.22
step:6000 loss:0.915663 accuracy: 58.18
step:6500 loss:0.908909 accuracy: 59.50
step:7000 loss:0.908536 accuracy: 59.63
step:7500 loss:0.909472 accuracy: 59.10
step:8000 loss:0.908312 accuracy: 59.63
step:8500 loss:0.908018 accuracy: 59.50
step:9000 loss:0.907985 accuracy: 59.76
step:9500 loss:0.907627 accuracy: 59.50
step:10000 loss:0.907957 accuracy: 59.23
step:10500 loss:0.907494 accuracy: 59.63
step:11000 loss:0.907859 accuracy: 59.23
step:11500 loss:0.907505 accuracy: 58.97
step:12000 loss:0.910241 accurac

In [102]:
to_pred['team_tie_prob']  = pred_df[0]
to_pred['team_1_prob']    = pred_df[1]
to_pred['team_2_prob']    = pred_df[2]

In [103]:
throne = peyton.Throne(username='JulienHeiduk', token="bcebc6d7-f224-4940-a816-74e3e6d4c34a")

# Submit predictions 
throne.competition('Italian Serie A').submit(to_pred)

{'success': True}

In [104]:
to_pred

,id,date,team_1_name,team_2_name,d_ability_1,d_ability_2,is_may,is_monday,is_friday,team_1_prob,team_2_prob,confidence,team_tie_prob,competition,year,month,day
0,5a2b1a2e977222252c6ceeb5,2017-12-22 18:00:00,Chievo,Bologna,-0.313977,-0.457664,-0.322762,-0.158932,10.621962,0.605896,0.218079,1.0,0.176025,Italian Serie A,2017,12,22
1,5a2b1a2e977222252c6ceeb6,2017-12-22 20:45:00,Cagliari,Fiorentina,-1.138282,-1.106496,-0.322762,-0.158932,10.621962,0.393947,0.423676,1.0,0.182377,Italian Serie A,2017,12,22
2,5a2c5a599772220818645341,2017-12-23 12:30:00,Lazio,Crotone,2.045917,2.014531,-0.322762,-0.158932,-0.094145,0.811358,0.032957,1.0,0.155684,Italian Serie A,2017,12,23
3,5a2c5a599772220818645342,2017-12-23 15:00:00,Genoa,Benevento,0.719251,0.645991,-0.322762,-0.158932,-0.094145,0.688043,0.116268,1.0,0.195689,Italian Serie A,2017,12,23
4,5a2c5a599772220818645343,2017-12-23 15:00:00,Napoli,Sampdoria,2.090347,2.239053,-0.322762,-0.158932,-0.094145,0.808093,0.037961,1.0,0.153945,Italian Serie A,2017,12,23
5,5a2c5a599772220818645344,2017-12-23 15:00:00,Sassuolo,Inter,-1.769815,-1.940056,-0.322762,-0.158932,-0.094145,0.187417,0.606469,1.0,0.206114,Italian Serie A,2017,12,23
6,5a2c5a599772220818645345,2017-12-23 15:00:00,Spal,Torino,-0.685119,-0.742198,-0.322762,-0.158932,-0.094145,0.357676,0.380419,1.0,0.261905,Italian Serie A,2017,12,23
7,5a2c5a599772220818645346,2017-12-23 15:00:00,Udinese,Verona,0.822499,0.736680,-0.322762,-0.158932,-0.094145,0.658612,0.122951,1.0,0.218437,Italian Serie A,2017,12,23
8,5a2c5a599772220818645347,2017-12-23 18:00:00,AC Milan,Atalanta,-0.314932,-0.355119,-0.322762,-0.158932,-0.094145,0.539347,0.256862,1.0,0.203791,Italian Serie A,2017,12,23
9,5a2c5a599772220818645348,2017-12-23 20:45:00,Juventus,AS Roma,0.489349,0.593815,-0.322762,-0.158932,-0.094145,0.660603,0.170092,1.0,0.169305,Italian Serie A,2017,12,23
